In [35]:
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
# load ascii text and covert to lowercase
filename = "AIW.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [33]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163815
Total Vocab:  60


In [5]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

In [7]:
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

163715

In [30]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(60,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(X, y, epochs=1, batch_size=128)

Epoch 1/1
163715/163715 [==============================] - 2373s 14ms/step - loss: 3.0373 - acc: 0.1791


In [36]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" of trials, “there was some attempts
at applause, which was immediately suppressed by the officers of "
 the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th the th

In [38]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(60,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(X, y, epochs=1, batch_size=128)

Epoch 1/1
163715/163715 [==============================] - 7853s 48ms/step - loss: 2.9942 - acc: 0.1900


In [39]:
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='relu'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()
model.fit(X, y, epochs=1, batch_size=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 100, 128)          66560     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100, 128)          0         
_________________________________________________________________
dense_16 (Dense)             (None, 100, 60)           7740      
Total params: 74,300
Trainable params: 74,300
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected dense_16 to have 3 dimensions, but got array with shape (163715, 60)